### Notebook for training used on the provided dataset


Using the NVIDIA architecture
This is mainly used for looking at the potential preprocessing steps, model architecture etc.

In [1]:
import os
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
from scipy import ndimage
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, Cropping2D
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
# import cv2

Using TensorFlow backend.


In [2]:
import os
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
from scipy import ndimage
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, Cropping2D
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
# import cv2

In [3]:
path_label = "data/driving_log.csv"
path_img = "data/IMG"
df = pd.read_csv(path_label)
labels = df["steering"].values

In [4]:
images = []
fnames = os.listdir(path_img)
# img = cv2.imread(os.path.join(path_img, fnames[0]), cv2.COLOR_BGR2RGB)
for fname in fnames:
    # only use CENTER images first
    if fname.startswith("center"):
        img = plt.imread(os.path.join(path_img, fname))
        images.append(img)
images = np.array(images)

In [5]:
X_train = images
y_train = labels
assert len(X_train) == len(y_train)

In [6]:
# define model using NVIDIA architecture
model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))  # normalize
# add cropping
model.add(Cropping2D(cropping=((70, 25), (0, 0))))
model.add(Conv2D(24,(5,5), strides=(2,2), activation="relu"))
model.add(Conv2D(36, (5,5), strides=(2,2), activation="relu"))
model.add(Conv2D(48, (5,5), strides=(2,2), activation="relu"))
model.add(Conv2D(64, (3,3), strides=(1,1), activation="relu"))
model.add(Conv2D(64, (3,3), strides=(1,1), activation="relu"))
model.add(Flatten())
model.add(Dense(1164))
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 31, 158, 24)       1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 77, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 37, 48)         43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 35, 64)         27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 33, 64)         36928     
__________

In [8]:
model.compile(loss=keras.losses.mse,
              optimizer=keras.optimizers.Adam())

In [9]:
model.fit(X_train, y_train, validation_split=0.2, batch_size=128, epochs=5, shuffle=True)

Train on 6428 samples, validate on 1608 samples
Epoch 1/5
6428/6428 [==============================] - 20s 3ms/step - loss: 0.0159 - val_loss: 0.0112
Epoch 2/5
6428/6428 [==============================] - 9s 1ms/step - loss: 0.0104 - val_loss: 0.0106
Epoch 3/5
6428/6428 [==============================] - 9s 1ms/step - loss: 0.0098 - val_loss: 0.0110
Epoch 4/5
6428/6428 [==============================] - 9s 1ms/step - loss: 0.0095 - val_loss: 0.0108
Epoch 5/5
6428/6428 [==============================] - 9s 1ms/step - loss: 0.0090 - val_loss: 0.0105


In [11]:
# model.save("model.h5")
model.save_weights("model.h5")

In [ ]:
K.clear_session()

In [10]:
# ToDos for first step

# write generator for iterative loading of images
# add regularization stuff (batch_norm, dropout)
# train and save model
# deploy model and test

#### Further Experimentations:


- different processing strategies in lambda layer
- Image augmentation (flipping)
- cropping images (top and bottom)
- use left and right images for recovery
- modify model architecture
- iterative, "transfer" learning approach (see https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/Behavioral+Cloning+Cheatsheet+-+CarND.pdf)
